In [1]:
import time
import pandas as pd
import DigiM_Execute as dme
import DigiM_Session as dms
import DigiM_Tool as dmt

# Hugging Faceのデータセット使う場合
import os
from dotenv import load_dotenv
from huggingface_hub import login
import pandas as pd

load_dotenv("system.env")
hf_token = os.getenv("HF_TOKEN")
login(token=hf_token)

# Hugging Faceのデータセットアドレス
splits = {'test': 'v2.0/test-00000-of-00001.parquet'}
hf_df = pd.read_parquet("hf://datasets/llm-jp/AnswerCarefully/" + splits["test"])

In [ ]:
#ベンチマークテストの設定
test_folder_path = "test/"
test_name = "バッチテスト"
test_file = "Sample_BatchTest.xlsx" #"16TEST_Animal.xlsx" "AnswerCarefullyTest.xlsx"
test_sheet_name = "Test"
raw_name_Q = "設問" 
raw_name_A = "デジタルコンサル"

#入力
#query = "以下の質問について「同意(+2)～否定(-2)」の間で回答してください。※回答は数値(+2～-2)のみを答えてください。\n"
query = "以下の質問について、簡潔に回答してください。"
uploaded_contents = []
situation = {}

#テスト結果と模範解答の比較
compare_flg = False #模範回答との比較を行うときはTrueに設定
raw_name_A_ref = "回答例"#"回答例"
raw_name_compare = "比較結果"
query_compare = """
次の二つのテキストについて比較を行い、以下を出力してください。\n
・評価：意見があっているか「合っている」「どちらともいえない」「合っていない」のいずれかから選択\n
・理由：評価を選択した理由を30文字程度
"""

#実行設定
agent_file = "agent_X0Sample.json"
add_knowledges = []
overwrite_items = {}
execution = {}
execution["MEMORY_USE"] = False
execution["MEMORY_SIMILARITY"] = False
execution["MAGIC_WORD_USE"] = False
execution["STREAM_MODE"] = False
execution["SAVE_DIGEST"] = False
execution["META_SEARCH"] = True
execution["RAG_QUERY_GENE"] = True

In [ ]:
# セッションの設定（新規でセッションIDを発番）
session_id = dms.set_new_session_id()
session_name = test_name
session = dms.DigiMSession(session_id, session_name)

service_info = {"SERVICE_ID":"Streamlit", "SERVICE_DATA":{}}
user_info = {"USER_ID":"WEB_DUMMY", "USER_DATA":{}}

# テストファイルを読み込んでループ
test_file_path = test_folder_path + test_file
mbti_sheet = pd.read_excel(test_file_path, sheet_name=test_sheet_name)
Q_no = 0
for index, row in mbti_sheet.iterrows():
    questionaire = str(row[raw_name_Q])
    user_input = query +"\n"+ questionaire
    response = ""
    for response_service_info, response_user_info, response_chunk in dme.DigiMatsuExecute_Practice(service_info, user_info, session_id, session_name, agent_file, user_input, uploaded_contents, situation, overwrite_items, add_knowledges, execution):
        response += response_chunk
    mbti_sheet.at[index, raw_name_A] = response
    Q_no += 1
    print(f"Q{Q_no}に回答:{response[:30]}")

    # 比較の実行
    if compare_flg:
        _, _, response_compare, _, _, _  = dmt.compare_texts({}, {}, raw_name_A_ref, str(row[raw_name_A_ref]), raw_name_A, response, query_compare)
        mbti_sheet.at[index, raw_name_compare] = response_compare
        print(f"Q{Q_no}との比較を完了:{response_compare[:30]}")
        
    time.sleep(3)

# テスト結果の出力
output_file = "output_"+ test_file
output_file_path = test_folder_path + output_file
mbti_sheet.to_excel(output_file_path, sheet_name=test_sheet_name, index=False)

print(f"テストが完了しました:{output_file_path}")

Q1に回答:私は「デジタルコンサル」といいます。東京在住のコンサルティン
Q2に回答:ワタシの出身地は設定されていないので、特定の出生地はありませ
Q3に回答:どちらかと言えばご飯派です！特においしいお米と和風のおかずが
Q4に回答:大勢の人といるのは好きですが、状況によりますね。仕事の場で多
Q5に回答:運転は得意というわけではなく、必要があればする程度です。都市
テストが完了しました:test/output_Sample_BatchTest.xlsx
